In [96]:
import pandas as pd, torch, random, re
import numpy as np
from nltk.tokenize import word_tokenize
from torchtext.data import Field, TabularDataset, Iterator, BucketIterator
from sklearn.model_selection import train_test_split
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from rnn import RNN_GRU
from torch import nn
from torch.optim import Adam
import torch.optim as optim


In [97]:
def split_data(csv_file):
    data = pd.read_csv(csv_file)[['genre','lyrics']].dropna()
    x = data.drop('genre',axis=1)
    y = data.genre
    X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.5)
    #add val set some time
    train = X_train.join(y_train,how='outer')
    test = X_test.join(y_test,how ='outer')    
    return train, test

In [98]:
train, test = split_data('data/lyrics.csv')

In [99]:
def get_genres(df):
    
    genres = []
    for i in train['genre']:
        if i not in genres:
            genres.append(i)
        else:
            continue
    
    
    return int2genre, genre2int

In [100]:
genres = ['Rock', 'Pop', 'R&B', 'Metal', 'Country', 'Folk', 
          'Hip-Hop', 'Indie', 'Jazz', 'Electronic']

In [101]:
train

,lyrics,genre
225267,",\n,\n,\n,\n! ! !\n,\n,\n, -\n! ! ! !",Rock
214193,A little gurl growing up\nNeeds some more atte...,Rock
63909,"I wake up, my eyes are playing tricks on me\na...",Rock
91004,Oooooooh\nBlack and orange stray cat sittin' o...,Rock
66408,"Lately, I find myself out gazing at stars\nHea...",Jazz
15219,Alanis Morissette & Ryan Adams\nRyan :\nI was ...,Rock
221792,Take him back Rachel\nIt's time to stop lying\...,Pop
36378,The room is empty\nThe lights are dim\nAnd my ...,Pop
38243,Had to triple back before i flipped out\nFeeli...,Other
146886,This is the moment\nThis is the day\nWhen I se...,Rock


In [102]:
def get_genre_counts(df):
    labels = []
    for i in train['genre']:
        labels.append(i)
        nr_genrs = Counter(labels)
    
    return nr_genrs

In [103]:
def gen_data_frames(df, genre2int, max_num):
     
    nr_training_examples = {'Rock'   : 0,
                        'Pop'        : 0,
                        'R&B'        : 0,
                        'Metal'      : 0,
                        'Country'    : 0,
                        'Folk'       : 0,
                        'Hip-Hop'    : 0,
                        'Indie'      : 0,
                        'Jazz'       : 0,
                        'Electronic' : 0
                        }
    
    index = []
    lyrics = []
    labels = []
    index = 0

    list_of_genres = list(nr_training_examples.keys())
    for i in df['genre']:
        if i in list_of_genres and nr_training_examples[i] != max_num:
            nr_training_examples[i] += 1
            lyrics.append(df.iloc[index, 0])
            labels.append(genre2int[df.iloc[index, 1]])
            index += 1
        else:
            index += 1
            continue
            
    df = pd.DataFrame(data=zip(lyrics,labels), columns=['lyrics', 'genre'])
    
    x = lyrics
    y = labels 
    return df, x, y
    
    

In [148]:
testing_df, x_test, y_test = gen_data_frames(test, genre2int, 100000)

KeyError: 'Other'

In [149]:
training_df, x_train, y_train = gen_data_frames(train, genre2int, 1000000)

KeyError: 'Other'

In [106]:
def expand_lyrics(df):
    song = []
    labels = []
    
    index = 0
    for i in df['lyrics']:
        lyric = (i.split('\n'))
        lyric = [(i,df.iloc[index,1]) for i in lyric]
        index += 1
        song += lyric
    return song

In [107]:
all_train_lyrics = expand_lyrics(training_df)


In [108]:
all_test_lyrics = expand_lyrics(testing_df)

In [109]:
train_df = pd.DataFrame(data=zip([i[0] for i in all_train_lyrics],[i[1] for i in all_train_lyrics]), columns=['lyrics', 'genre'])

In [110]:
test_df = pd.DataFrame(data=zip([i[0] for i in all_test_lyrics],[i[1] for i in all_train_lyrics]), columns=['lyrics', 'genre'])

In [111]:
train_df

,lyrics,genre
0,",",0
1,",",0
2,",",0
3,",",0
4,! ! !,0
5,",",0
6,",",0
7,", -",0
8,! ! ! !,0
9,A little gurl growing up,0


In [112]:
def gen_data_frames(df, encoder, max_num):
     
    nr_training_examples = {'Rock'   : 0,
                        'Pop'        : 0,
                        'R&B'        : 0,
                        'Metal'      : 0,
                        'Country'    : 0,
                        'Folk'       : 0,
                        'Hip-Hop'    : 0,
                        'Indie'      : 0,
                        'Jazz'       : 0,
                        'Electronic' : 0
                        }
    index = []
    lyrics = []
    labels = []
    index = 0
    list_of_genres = list(nr_training_examples.keys())
    for i in df['genre']:
        if encoder[i] in list_of_genres and nr_training_examples[encoder[i]] < max_num:
            nr_training_examples[encoder[i]] += 1
            lyrics.append(df.iloc[index, 0])
            labels.append(encoder[df.iloc[index, 1]])
            index += 1
        else:
            index += 1
            continue
    
    df = pd.DataFrame(data=zip(lyrics,labels), columns=['lyrics', 'genre'])
    
    x = lyrics
    y = labels 
    return df, x, y

In [150]:
train_dafray, x_train, y_train = gen_data_frames(train_df, int2genre, 100000)



In [152]:
test_dafray, x_test, y_test = gen_data_frames(test_df, int2genre, 10000)


In [120]:
import bcolz

ModuleNotFoundError: No module named 'bcolz'

In [118]:
words = []
idx = 0
word2idx = {}
vectors = bcolz.carray(np.zeros(1), rootdir=f'{glove_path}/6B.50.dat', mode='w')

with open(f'{glove_path}/glove.6B.50d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        words.append(word)
        word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)
    
vectors = bcolz.carray(vectors[1:].reshape((400000, 50)), rootdir=f'{glove_path}/6B.50.dat', mode='w')
vectors.flush()
pickle.dump(words, open(f'{glove_path}/6B.50_words.pkl', 'wb'))
pickle.dump(word2idx, open(f'{glove_path}/6B.50_idx.pkl', 'wb'))

NameError: name 'bcolz' is not defined

In [ ]:
train_dafray.to_csv('training.csv')

In [ ]:
test_dafray.to_csv('testing.csv')

In [2]:
train_dafray = pd.read_csv('training.csv')

In [3]:
test_dafray = pd.read_csv('testing.csv')

In [153]:
x_train = [i for i in train_dafray['lyrics']]

In [154]:
x_test = [i for i in test_dafray['lyrics']]

In [155]:
y_train = [i for i in train_dafray['genre']]

In [156]:
y_test = [i for i in train_dafray['genre']]

In [157]:
def pre_process(train_data):
    #word_counts = {}
    vocab = set()
    training_data = []
    
    index = 0
    for i in train_data:
        tokenized = word_tokenize(i)
        training_data.append(tokenized)
        vocab.update(set(tokenized))
        #word_counts.update(dict(Counter(tokenized)))
        
        
    int2word = dict(enumerate(vocab))
    word2int = {word : num for num, word in int2word.items()}
    
    return training_data, word2int, word_counts
    
    

In [158]:
pre_process_x_train, vocab, word_counts = pre_process(x_train)

In [159]:
pre_process_x_test, vocab, word_counts = pre_process(x_test)

In [160]:
def encode_seq(data, char2int):
    vectors = []
    
    for words in data:
        vector = []
        for word in words[:15]:
            if word in vocab:
                vector.append(char2int[word])
            else:
                vector.append(random.randint(0, len(vocab) - 1))
        vectors.append(torch.LongTensor(vector))
        
     
    return pad_sequence(vectors, batch_first=True, padding_value=0)
        

In [161]:
train_padded = encode_seq(pre_process_x_train, vocab)

In [162]:
test_padded = encode_seq(pre_process_x_test, vocab)

In [163]:
genres = ['Rock', 'Pop', 'R&B', 'Metal', 'Country', 'Folk', 
      'Hip-Hop', 'Indie', 'Jazz', 'Electronic']       
int2genre = dict(enumerate(genres))
genre2int = {genre : num for num, genre in int2genre.items()}

In [164]:
labels = [genre2int[i] for i in y_train]

In [165]:
test_labels = [genre2int[i] for i in y_test]

In [166]:
from torch.utils.data import Dataset

class ProjDataset(Dataset):
    def __init__(self, train_x, train_y):
        self.train_x = train_x
        self.train_y = train_y

    def __getitem__(self, index):
        return (self.train_x[index], self.train_y[index])

    def __len__(self):
        return len(self.train_x)

        


In [167]:
train_dataset = ProjDataset(train_padded, labels)

In [168]:
test_dataset = ProjDataset(test_padded, test_labels)

In [169]:
train_dl = DataLoader(train_dataset, batch_size=100, shuffle=True)

In [170]:
device=torch.device('cuda:02')

In [171]:
model = RNN_GRU(vocab_size=len(vocab), seq_len=15,
                   input_size=15, hidden_size=128,
                   num_layers=2, output_size=len(genre2int),
                   device=device, dropout=0.0)

In [172]:
print(model)

RNN_GRU(
  (emb): Embedding(40632, 15)
  (gru): GRU(15, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=1920, out_features=10, bias=True)
)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)
model.train()
model = model.to(device)
print('Training')
epoch_nr = 0
EPOCH = list(range(40))
tenp = len(train_dl) / 10
batch_size = 100

for epoch in EPOCH:

    epoch_nr += 1
    epoch_loss = []
    h = model.init_hidden(batch_size)
    count = 0
    percent = 0

    for (x,y) in train_dl:
        if len((x,y)) != batch_size:
            continue
            
        count +=1
        x = x.to(device)
        
        y = y.to(device)
        
        optimizer.zero_grad()
        h = h.data
        
        out, h = model(x, h, device)
    
        loss = criterion(out, y)
        loss.backward()
        epoch_loss.append(loss.item())
        optimizer.step()
        if count % tenp == 0:
            percent += 10
            print("Training {}% complete".format(round(percent,1)))
        
    
    #avg_loss = sum(epoch_loss) / len(epoch_loss)
    #print("Average loss at epoch %d: %.7f" % (epoch_nr, avg_loss))

Training


In [147]:
correct = 0
percent = 0
example = 0
tenp = len(test_dataset) / 10
count = 0
for x, y in test_dataset:
    example +=1
    hidden_layer = model.init_hidden(1).to(device)
    x = x.unsqueeze(0)
    
    count += 1
    prediction = model(x.to(device), hidden_layer, device)
    _, indeces = torch.max(prediction[0].data, dim=1)
    if indeces[0].item() == y:
        correct += 1
    

    if count % tenp == 0:
        percent += 10
        print('Accuracy after {}% tested: {}'.format(percent, (correct / example * 100)))



Accuracy after 10% tested: 10.51
Accuracy after 20% tested: 9.945
Accuracy after 30% tested: 10.236666666666666
Accuracy after 40% tested: 10.535
Accuracy after 50% tested: 10.367999999999999
Accuracy after 60% tested: 10.313333333333333
Accuracy after 70% tested: 10.194285714285714
Accuracy after 80% tested: 10.14
Accuracy after 90% tested: 10.174444444444445


KeyboardInterrupt: 